In [5]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
options.add_experimental_option("excludeSwitches", ["enable-logging"])

driver = webdriver.Chrome('chromedriver.exe', options=options)
driver.implicitly_wait(3)
url = 'https://search.shopping.naver.com/catalog/11291155550?query=%ED%82%A4%EB%B3%B4%EB%93%9C&NaPm=ct%3Dl18sc69k%7Cci%3D8e5380ba416ac7409569a558d66f11dfef15e2ff%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D1d342903cb5cc9ac8e2fbc776a1d9709d34f4b16'
#section_review > div.pagination_pagination__2M9a4 > a.pagination_now__gZWGP

C:\Users\Main\AppData\Local\Temp/ipykernel_11652/1883807372.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe', options=options)


1번 버튼: //*[@id="section_review"]/div[3]/a[1] 
10번 버튼: //*[@id="section_review"]/div[3]/a[10] 
1~10번 페이지의 다음 버튼: //*[@id="section_review"]/div[3]/a[11]
    
11번 버튼: //*[@id="section_review"]/div[3]/a[2] 
20번 버튼: //*[@id="section_review"]/div[3]/a[11] 
11번 페이지 이후부터 다음 버튼: //*[@id="section_review"]/div[3]/a[12] (이전 버튼 때문에 한 칸씩 밀림)

In [6]:
# 빈 리스트 생성
star_list = []
review_list = []
review_date_list = []
review_mall_list = []
review_pic_list = []

In [7]:
driver.get(url)

# 한자리수 리뷰페이지의 버튼 xpath와 두자리수 리뷰페이지의 버튼 xpath가 다름.
for i in range(1, 11):
    time.sleep(0.5)
    # 리뷰페이지 버튼 누르기 (1~10번 순으로 누르기)
    driver.find_element_by_xpath('//*[@id="section_review"]/div[3]/a[{}]'.format(i)).click()

    time.sleep(0.5)

    # 별점 리스트에 저장
    star = driver.find_elements_by_css_selector('ul.reviewItems_list_review__1sgcJ li span.reviewItems_average__16Ya-')
    for st in star:
        star_list.append(st.text)

    time.sleep(1)
    
    # 리뷰 작성 날짜 리스트에 저장
    a = 1
    for _ in range(20):
        try:
            review_date = driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li[{}]/div[1]/span[4]'.format(a))
            for dates in review_date:
                review_date_list.append(dates.text)
            a += 1
        except:
            a += 1
    
    time.sleep(0.5)
    # 리뷰 작성한 플랫폼 리스트에 저장
    k = 1
    for _ in range(20):
        try:
            review_mall = driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li[{}]/div[1]/span[2]'.format(k))
            for malls in review_mall:
                review_mall_list.append(malls.text)
            k += 1
        except:
            k += 1

    time.sleep(0.5)
    
    # 리뷰내용(텍스트) 리스트에 저장
    reviews = driver.find_elements_by_css_selector('ul.reviewItems_list_review__1sgcJ div.reviewItems_review_text__2Bwpa p.reviewItems_text__XIsTc')
    for rv in reviews:
        review_list.append(rv.text)

    time.sleep(0.5)
    
    # 이미지 개수 가져오기
    for p in range(1, 21):
        review_pic = driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li[{}]/div[2]/div[2]'.format(p))
    
        if len(review_pic) == 0:
            review_pic_list.append('0')
        for pi in review_pic:
            if len(pi.text) > 0:
                review_pic_list.append(pi.text)
            elif len(pi.text) == 0:
                review_pic_list.append('1')
        time.sleep(0.3)
        
# 10페이지에서 '다음' 버튼 눌러서 11페이지로 가게 하기
driver.find_element(by=By.XPATH, value='//*[@id="section_review"]/div[3]/a[11]').click()

for pages in range(9):
    for i in range(2, 12):
        # 리뷰 페이지 버튼 누르기 (11 ~ 20, 21 ~ 30, ..., 91 ~ 100)
        driver.find_element_by_xpath('//*[@id="section_review"]/div[3]/a[{}]'.format(i)).click()

        time.sleep(0.5)
        
        # 별점 리스트에 저장
        star = driver.find_elements_by_css_selector('ul.reviewItems_list_review__1sgcJ li span.reviewItems_average__16Ya-')
        for st in star:
            star_list.append(st.text)

        time.sleep(0.5)

        # 리뷰 작성 날짜 리스트에 저장
        a = 1
        for _ in range(20):
            try:
                review_date = driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li[{}]/div[1]/span[4]'.format(a))
                for dates in review_date:
                    review_date_list.append(dates.text)
                a += 1
            except:
                a += 1
        
        # 리뷰 작성한 플랫폼 리스트에 저장
        k = 1
        for _ in range(20):
            try:
                review_mall = driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li[{}]/div[1]/span[2]'.format(k))
                for malls in review_mall:
                    review_mall_list.append(malls.text)
                k += 1
            except:
                k += 1

        time.sleep(0.5)
        
        # 리뷰내용(텍스트) 리스트에 저장
        reviews = driver.find_elements_by_css_selector('ul.reviewItems_list_review__1sgcJ div.reviewItems_review_text__2Bwpa p.reviewItems_text__XIsTc')
        for rv in reviews:
            review_list.append(rv.text)

        time.sleep(0.5)
        
        # 이미지 개수 가져오기 
        for p in range(1, 21):
            review_pic = driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li[{}]/div[2]/div[2]'.format(p))
    
            if len(review_pic) == 0:
                review_pic_list.append('0')
            for pi in review_pic:
                if len(pi.text) > 0:
                    review_pic_list.append(pi.text)
                elif len(pi.text) == 0:
                    review_pic_list.append('1')
            time.sleep(0.3)
        
    # 20 -> 21페이지로 넘어가는 다음 버튼 클릭
    driver.find_element(by=By.XPATH, value='//*[@id="section_review"]/div[3]/a[12]').click()

C:\Users\Main\AppData\Local\Temp/ipykernel_11652/2975513430.py:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="section_review"]/div[3]/a[{}]'.format(i)).click()
C:\Users\Main\AppData\Local\Temp/ipykernel_11652/2975513430.py:12: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  star = driver.find_elements_by_css_selector('ul.reviewItems_list_review__1sgcJ li span.reviewItems_average__16Ya-')
C:\Users\Main\AppData\Local\Temp/ipykernel_11652/2975513430.py:22: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  review_date = driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li[{}]/div[1]/span[4]'.format(a))
C:\Users\Main\AppData\Local\Temp/ipykernel_11652/2975513430.py:34: DeprecationWarning: find_elements_by_xpath i

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00E37413+2389011]
	Ordinal0 [0x00DC9F61+1941345]
	Ordinal0 [0x00CBC658+837208]
	Ordinal0 [0x00CBF064+847972]
	Ordinal0 [0x00CBEF22+847650]
	Ordinal0 [0x00CBF1B0+848304]
	Ordinal0 [0x00CEF6AB+1046187]
	Ordinal0 [0x00CED4F4+1037556]
	Ordinal0 [0x00CEB5EB+1029611]
	Ordinal0 [0x00CEA359+1024857]
	Ordinal0 [0x00CDFF93+982931]
	Ordinal0 [0x00D0414C+1130828]
	Ordinal0 [0x00CDF974+981364]
	Ordinal0 [0x00D04364+1131364]
	Ordinal0 [0x00D14302+1196802]
	Ordinal0 [0x00D03F66+1130342]
	Ordinal0 [0x00CDE546+976198]
	Ordinal0 [0x00CDF456+980054]
	GetHandleVerifier [0x00FE9632+1727522]
	GetHandleVerifier [0x0109BA4D+2457661]
	GetHandleVerifier [0x00ECEB81+569713]
	GetHandleVerifier [0x00ECDD76+566118]
	Ordinal0 [0x00DD0B2B+1968939]
	Ordinal0 [0x00DD5988+1989000]
	Ordinal0 [0x00DD5A75+1989237]
	Ordinal0 [0x00DDECB1+202

In [8]:
len(star_list)

1800

In [9]:
all_review_data = {
'리뷰 날짜': review_date_list,
'리뷰 쇼핑몰' : review_mall_list,
'리퓨 평점' : star_list,
'리뷰 내용' : review_list,
'이미지 수' : review_pic_list
}

df = pd.DataFrame(all_review_data)
print(df)

          리뷰 날짜   리뷰 쇼핑몰 리퓨 평점  \
0     20.09.14.       옥션   평점5   
1     20.02.27.     11번가   평점4   
2     21.01.15.     11번가   평점4   
3     20.01.17.     11번가   평점5   
4     19.05.29.       옥션   평점5   
...         ...      ...   ...   
1795  21.03.04.   N정품스토어   평점5   
1796  21.01.05.   N정품스토어   평점5   
1797  21.04.25.  앱코공식스토어   평점1   
1798  20.10.27.      컴퓨존   평점4   
1799  20.06.08.  앱코공식스토어   평점5   

                                                  리뷰 내용       이미지 수  
0     일단 배송도 빨라서 넘좋았습니다\n토요일날주문해서 월요일날 도착했나? 그럴겁니다\n...           0  
1     기존에 사용하던 키보드는 8만원정도 하는데 청축을 사용했는데요. 키 버튼 하나를 누...           0  
2     갑작스럽게 기계식 키보드가 필요해서 주문했습니다. 일단 배송은 매우 빠르게 출고해주...           0  
3     기계식 키보드를 처음 써보는데 그동안 많이 궁금했습니다. 마침 이 제품이 절반정도 ...           0  
4     갈축과 적축 두개를 구매했습니다.갈축은 불량이 걸려서 현재 교환요청 중이구요,적축은...  4\n개의 포토리뷰  
...                                                 ...         ...  
1795  동생 키보드 멤브레인 쓰다가 기계식으로 바꿔주는데 할인 하길래 바로 사버렸어요~\n...           1  
1796  컴퓨터 잘 아는 분이 사용 권장해주셔서 구매했

In [10]:
df.head()

,리뷰 날짜,리뷰 쇼핑몰,리퓨 평점,리뷰 내용,이미지 수
0,20.09.14.,옥션,평점5,일단 배송도 빨라서 넘좋았습니다\n토요일날주문해서 월요일날 도착했나? 그럴겁니다\n...,0
1,20.02.27.,11번가,평점4,기존에 사용하던 키보드는 8만원정도 하는데 청축을 사용했는데요. 키 버튼 하나를 누...,0
2,21.01.15.,11번가,평점4,갑작스럽게 기계식 키보드가 필요해서 주문했습니다. 일단 배송은 매우 빠르게 출고해주...,0
3,20.01.17.,11번가,평점5,기계식 키보드를 처음 써보는데 그동안 많이 궁금했습니다. 마침 이 제품이 절반정도 ...,0
4,19.05.29.,옥션,평점5,"갈축과 적축 두개를 구매했습니다.갈축은 불량이 걸려서 현재 교환요청 중이구요,적축은...",4\n개의 포토리뷰


In [12]:
import pickle 

with open('ml_data.pickle','wb') as pickle_filename:
    pickle.dump(df, pickle_filename)